In [13]:
from rllib_setup import get_env
import ray
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.algorithms.dqn import DQNConfig

alg_name = "DQN"
env_name = "VJS"
register_env(env_name,lambda config: get_env())

test_env = get_env()
obs_space = test_env.observation_space
act_space = test_env.action_space
def policies(agent_ids):
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }


In [14]:
config = (
    DQNConfig()
    .environment(env=env_name, disable_env_checking=True)
    .rollouts(num_rollout_workers=10, rollout_fragment_length=300,create_env_on_local_worker=True,num_envs_per_worker=1,)
    .training(
        train_batch_size=200,
    )
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .framework(framework="torch")
    .exploration(
        exploration_config={
            # The Exploration class to use.
            "type": "EpsilonGreedy",
            # Config for the Exploration class' constructor:
            "initial_epsilon": 0.1,
            "final_epsilon": 0.0,
            "epsilon_timesteps": 100000,  # Timesteps over which to anneal epsilon.
        }
    )
)


In [15]:
algo = config.build()

/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:442: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future 

In [16]:
for j in range(1):
    for i in range(10):
        info = algo.train()
    print(info['info']['learner']['Machine_0']['mean_td_error'])



-1923.1690673828125


In [17]:
for i in range(10):
    info = algo.evaluate()
    
info

{'evaluation': {'sampler_results': {'episode_reward_max': 21025115.43749996,
   'episode_reward_min': 21025115.43749996,
   'episode_reward_mean': 21025115.43749996,
   'episode_len_mean': 4743.0,
   'episode_media': {},
   'episodes_this_iter': 1,
   'policy_reward_min': {'Machine_5': 3852487.125000001,
    'Machine_2': 2610096.0750000016,
    'Machine_3': -1014.75,
    'Machine_6': 1017248.2874999996,
    'Machine_7': 5439752.1,
    'Machine_4': 1507001.0624999993,
    'Machine_8': 242538.41250000006,
    'Machine_1': 2613772.6875000005,
    'Machine_0': 530206.7625,
    'Machine_9': 1435930.5375000003,
    'Machine_10': 529230.3750000007,
    'Machine_11': 1247866.7625},
   'policy_reward_max': {'Machine_5': 3852487.125000001,
    'Machine_2': 2610096.0750000016,
    'Machine_3': -1014.75,
    'Machine_6': 1017248.2874999996,
    'Machine_7': 5439752.1,
    'Machine_4': 1507001.0624999993,
    'Machine_8': 242538.41250000006,
    'Machine_1': 2613772.6875000005,
    'Machine_0': 530

(raylet) [2023-09-29 11:15:17,588 E 881477 881477] (raylet) node_manager.cc:3069: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: e8c50b41e1219f6981aee0ea755f4b41bc398917a8b23b26abb70544, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-09-29 11:16:17,589 E 881477 881477] (raylet) node_manager.cc:3069: 4 Workers (tasks / actors) killed d